# Monte Carlo Market Risk Engine
## Professional Risk Analysis Report

**Author:** Rivan Avinash Shetty  
**Model Type:** 1-Day Trading Desk Risk Model  
**Engine:** Correlated Monte Carlo Simulation (100,000 paths)  
**Extensions:** Student-t Heavy-Tailed MC, Backtesting, Stress Testing  

---

## Executive Summary

This report presents a comprehensive market risk analysis for a diversified 5-asset portfolio spanning equities, fixed income, and commodities. The risk engine implements **four** complementary VaR methodologies — Historical Simulation, Parametric (Variance-Covariance), Gaussian Monte Carlo, and **Student-t Monte Carlo** — validated through rolling-window backtesting and stress-tested under extreme volatility and correlation scenarios.

**Key Findings:**
- Portfolio composition: SPY (30%), QQQ (20%), JPM (15%), TLT (20%), GLD (15%)
- 99% 1-day VaR and Expected Shortfall computed across all four models
- **Student-t MC** captures heavy tails and produces higher VaR/ES than the Gaussian MC — more realistic for extreme events
- Backtesting validates model adequacy via Kupiec proportion-of-failures test
- Stress testing quantifies risk amplification under volatility doubling and correlation breakdown

### Regulatory Context

Under **Basel III / FRTB**, the regulatory standard shifted from VaR to **Expected Shortfall (ES)** because VaR does not capture the severity of tail losses. This engine computes both metrics, enabling a direct comparison of their behaviour across different distributional assumptions.

---
## 1. Setup & Configuration

In [ ]:
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
from pathlib import Path

warnings.filterwarnings('ignore')

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.portfolio import (
    fetch_data, load_data, compute_log_returns,
    define_weights, compute_portfolio_returns,
    get_portfolio_summary, DEFAULT_TICKERS,
)
from src.statistics import (
    get_all_statistics, compute_rolling_volatility,
)
from src.risk_metrics import historical_risk_metrics, parametric_risk_metrics
from src.monte_carlo import run_monte_carlo_engine
from src.student_t_mc import fit_degrees_of_freedom, run_student_t_mc_engine
from src.backtesting import run_full_backtest
from src.stress_testing import full_stress_analysis

# Configuration
DATA_PATH = PROJECT_ROOT / 'data' / 'raw_prices.csv'
NUM_SIMULATIONS = 100_000
RANDOM_SEED = 42

print('Environment ready.')
print(f'Project root: {PROJECT_ROOT}')

---
## 2. Data Acquisition & Portfolio Construction

### 2.1 Asset Selection

| Asset | Ticker | Weight | Rationale |
|-------|--------|--------|-----------|
| S&P 500 ETF | SPY | 30% | Broad market exposure |
| Nasdaq 100 ETF | QQQ | 20% | High-beta tech sector |
| JPMorgan Chase | JPM | 15% | Financial sector |
| 20+ Year Treasury | TLT | 20% | Duration / interest rate risk |
| Gold ETF | GLD | 15% | Safe-haven diversifier |

Cross-asset covariance structure is critical — this portfolio includes assets with low or negative correlations to capture meaningful diversification effects.

In [ ]:
# Load or fetch price data
if DATA_PATH.exists():
    prices = load_data(str(DATA_PATH))
    print(f'Loaded cached data: {prices.shape[0]} observations')
else:
    prices = fetch_data(save_path=str(DATA_PATH))
    print(f'Fetched fresh data: {prices.shape[0]} observations')

print(f'\nDate range: {prices.index[0].date()} to {prices.index[-1].date()}')
print(f'Assets: {list(prices.columns)}')
prices.tail()

In [ ]:
# Compute log returns and portfolio
# Pass prices.columns (alphabetical from yfinance) so weights are correctly aligned
weights = define_weights(tickers=list(prices.columns))
log_returns = compute_log_returns(prices)
port_returns = compute_portfolio_returns(log_returns, weights)

# Portfolio summary
summary = get_portfolio_summary(prices, weights)

print('Weight–Asset Alignment:')
for t, w in zip(prices.columns, weights):
    print(f'  {t}: {w:.2f}')
print()
print('Portfolio Summary')
print('=' * 45)
for k, v in summary.items():
    print(f'  {k:.<35} {v:>10.4f}')


### 2.2 Return Distributions

Log returns are computed as:

$$r_t = \ln\left(\frac{P_t}{P_{t-1}}\right)$$

The portfolio return at each time step is:

$$R_p = \mathbf{w}^T \mathbf{R}$$

In [ ]:
# Visualize individual asset return distributions
fig, axes = plt.subplots(2, 3, figsize=(16, 9))
axes = axes.flatten()

for i, col in enumerate(log_returns.columns):
    axes[i].hist(log_returns[col], bins=80, density=True, alpha=0.7,
                 color='steelblue', edgecolor='none')
    axes[i].set_title(f'{col} Daily Returns', fontsize=12, fontweight='bold')
    axes[i].axvline(0, color='black', linewidth=0.8, linestyle='--')
    axes[i].set_xlabel('Log Return')

# Portfolio returns in last subplot
axes[-1].hist(port_returns, bins=80, density=True, alpha=0.7,
              color='darkred', edgecolor='none')
axes[-1].set_title('Portfolio Return', fontsize=12, fontweight='bold')
axes[-1].axvline(0, color='black', linewidth=0.8, linestyle='--')
axes[-1].set_xlabel('Log Return')

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'figures' / 'return_distributions.png',
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 3. Statistical Estimation

### Key Estimators

**Mean vector:**
$$\boldsymbol{\mu} = E[\mathbf{r}]$$

**Covariance matrix:**
$$\Sigma = E\left[(\mathbf{r} - \boldsymbol{\mu})(\mathbf{r} - \boldsymbol{\mu})^T\right]$$

**Portfolio variance:**
$$\sigma_p^2 = \mathbf{w}^T \Sigma \mathbf{w}$$

In [ ]:
# Compute all statistics
mu, cov, corr, port_stats = get_all_statistics(log_returns, weights)

print('Daily Mean Return Vector')
print('=' * 35)
for i, ticker in enumerate(prices.columns):
    print(f'  {ticker}: {mu[i]:>12.6f}')

print(f'\nPortfolio Statistics')
print('=' * 35)
for k, v in port_stats.items():
    print(f'  {k:.<30} {v:>12.8f}')

In [ ]:
# Covariance matrix
print('Covariance Matrix (×10⁴ for readability):\n')
cov_df = pd.DataFrame(cov * 1e4, index=prices.columns, columns=prices.columns)
print(cov_df.to_string(float_format=lambda x: f'{x:.4f}'))

In [ ]:
# Correlation heatmap
fig, ax = plt.subplots(figsize=(9, 7))
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
sns.heatmap(corr, mask=mask, annot=True, fmt='.3f', cmap='RdYlBu_r',
            center=0, vmin=-1, vmax=1, square=True, linewidths=0.5,
            xticklabels=prices.columns, yticklabels=prices.columns, ax=ax,
            cbar_kws={'shrink': 0.8, 'label': 'Correlation'})
ax.set_title('Asset Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'figures' / 'correlation_heatmap.png',
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 4. Risk Model Results

Four VaR methodologies are implemented and compared:

### Model 1: Historical Simulation
- Distribution-free — uses empirical quantiles of realized losses
- No parametric assumptions, but backward-looking

### Model 2: Parametric (Variance-Covariance)
- Assumes Gaussian returns: $\text{VaR}_\alpha = z_\alpha \cdot \sigma_p - \mu_p$
- Fast, but underestimates tail risk

### Model 3: Gaussian Monte Carlo (Flagship)
- Cholesky decomposition: $\Sigma = \mathbf{L}\mathbf{L}^T$
- Correlated simulation: $\mathbf{R} = \boldsymbol{\mu} + \mathbf{L}\mathbf{Z}$, where $\mathbf{Z} \sim N(\mathbf{0}, \mathbf{I})$
- 100,000 paths — full P&L distribution

### Model 4: Student-t Monte Carlo (Heavy-Tailed Extension)
- Replaces Gaussian innovations with Student-t: $Z \sim t(\nu)$
- Degrees of freedom $\nu$ fitted via MLE on historical returns
- Captures fat tails: extreme moves are more probable than under Gaussian
- When $\nu \to \infty$, converges to the Gaussian MC (nested model)

In [ ]:
# Historical VaR
hist_metrics = historical_risk_metrics(port_returns)

# Parametric VaR
param_metrics = parametric_risk_metrics(
    port_stats['portfolio_mean'],
    port_stats['portfolio_std'],
)

# Gaussian Monte Carlo VaR
mc_results = run_monte_carlo_engine(
    mu, cov, weights, NUM_SIMULATIONS, RANDOM_SEED
)
mc_metrics = {
    'mc_var_95': mc_results['var_95'],
    'mc_var_99': mc_results['var_99'],
    'mc_es_95': mc_results['es_95'],
    'mc_es_99': mc_results['es_99'],
}

# Student-t Monte Carlo VaR
print('Fitting Student-t degrees of freedom via MLE...')
df_t, loc_t, scale_t = fit_degrees_of_freedom(port_returns.values)
print(f'  Fitted ν = {df_t:.2f}  (lower ν → heavier tails)')
print(f'  Location = {loc_t:.6f}, Scale = {scale_t:.6f}')

t_results = run_student_t_mc_engine(
    mu, cov, weights, df_t, NUM_SIMULATIONS, RANDOM_SEED
)
t_metrics = {
    't_var_95': t_results['var_95'],
    't_var_99': t_results['var_99'],
    't_es_95': t_results['es_95'],
    't_es_99': t_results['es_99'],
}

print(f'\nAll 4 models computed ({NUM_SIMULATIONS:,} simulations for MC engines).')

In [ ]:
# Comparison table — all 4 models
comparison = pd.DataFrame({
    'Model': ['Historical', 'Parametric (Gaussian)',
              'Monte Carlo (Gaussian)', f'Monte Carlo (Student-t, ν={df_t:.1f})'],
    '95% VaR': [
        hist_metrics['hist_var_95'],
        param_metrics['param_var_95'],
        mc_metrics['mc_var_95'],
        t_metrics['t_var_95'],
    ],
    '99% VaR': [
        hist_metrics['hist_var_99'],
        param_metrics['param_var_99'],
        mc_metrics['mc_var_99'],
        t_metrics['t_var_99'],
    ],
    '95% ES': [
        hist_metrics['hist_es_95'],
        param_metrics['param_es_95'],
        mc_metrics['mc_es_95'],
        t_metrics['t_es_95'],
    ],
    '99% ES': [
        hist_metrics['hist_es_99'],
        param_metrics['param_es_99'],
        mc_metrics['mc_es_99'],
        t_metrics['t_es_99'],
    ],
})

comparison.set_index('Model', inplace=True)

# Format as percentages for display
styled = comparison.style.format('{:.4%}').set_caption(
    'VaR & Expected Shortfall Comparison (1-Day Horizon)'
).set_properties(**{'text-align': 'right'})
styled

In [ ]:
# Monte Carlo P&L distribution
fig, ax = plt.subplots(figsize=(14, 7))

pnl = mc_results['portfolio_pnl']
ax.hist(pnl, bins=300, density=True, color='steelblue', alpha=0.7, edgecolor='none',
        label='Simulated P&L')

ax.axvline(-mc_results['var_95'], color='#ff7f0e', lw=2, ls='--',
           label=f"95% VaR = {mc_results['var_95']:.4%}")
ax.axvline(-mc_results['var_99'], color='#d62728', lw=2, ls='--',
           label=f"99% VaR = {mc_results['var_99']:.4%}")
ax.axvline(-mc_results['es_99'], color='#9467bd', lw=2, ls=':',
           label=f"99% ES = {mc_results['es_99']:.4%}")

import matplotlib.ticker as mtick
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_xlabel('Portfolio Return', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Monte Carlo Simulated 1-Day P&L Distribution', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'figures' / 'mc_pnl_distribution.png',
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# Left-tail zoom
fig, ax = plt.subplots(figsize=(14, 7))

cutoff = np.percentile(pnl, 10)
tail_data = pnl[pnl <= cutoff]

ax.hist(tail_data, bins=150, density=True, color='steelblue', alpha=0.7,
        edgecolor='none', label='Left Tail P&L')
ax.axvline(-mc_results['var_95'], color='#ff7f0e', lw=2, ls='--',
           label=f"95% VaR = {mc_results['var_95']:.4%}")
ax.axvline(-mc_results['var_99'], color='#d62728', lw=2, ls='--',
           label=f"99% VaR = {mc_results['var_99']:.4%}")
ax.axvline(-mc_results['es_99'], color='#9467bd', lw=2, ls=':',
           label=f"99% ES = {mc_results['es_99']:.4%}")
ax.axvspan(tail_data.min(), -mc_results['var_99'], alpha=0.15, color='#d62728',
           label='Tail Loss Region')

ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_xlabel('Portfolio Return', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Left Tail Zoom — VaR & Expected Shortfall', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'figures' / 'left_tail_zoom.png',
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

### 4.1 Gaussian vs Student-t — Fat Tail Impact

The key insight: **the Student-t distribution assigns more probability mass to extreme events** (tail returns beyond 3σ). This is critical because:

1. Real financial returns exhibit **excess kurtosis** — extreme moves happen more often than a Normal predicts
2. The Gaussian assumption causes VaR to **systematically underestimate** tail risk (confirmed by our backtesting below)
3. The fitted degrees of freedom $\nu$ directly quantify *how far from Gaussian* the returns are — lower $\nu$ means fatter tails

The chart below overlays both distributions side-by-side. Notice how the Student-t (red) has more mass in the far left tail, producing higher VaR and especially higher ES.

In [ ]:
# Gaussian vs Student-t distribution overlay
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

g_pnl = mc_results['portfolio_pnl']
t_pnl = t_results['portfolio_pnl']

# Left panel: Full distribution overlay
ax = axes[0]
ax.hist(g_pnl, bins=250, density=True, alpha=0.55, color='steelblue',
        edgecolor='none', label='Gaussian MC')
ax.hist(t_pnl, bins=250, density=True, alpha=0.55, color='#d62728',
        edgecolor='none', label=f'Student-t MC (ν={df_t:.1f})')
ax.axvline(-mc_results['var_99'], color='steelblue', lw=2, ls='--',
           label=f"Gauss 99% VaR = {mc_results['var_99']:.4%}")
ax.axvline(-t_results['var_99'], color='#d62728', lw=2, ls='--',
           label=f"t 99% VaR = {t_results['var_99']:.4%}")
ax.set_xlabel('Portfolio Return', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Full P&L Distribution — Gaussian vs Student-t', fontsize=13, fontweight='bold')
ax.legend(fontsize=9.5, loc='upper right')
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Right panel: Left-tail zoom
ax2 = axes[1]
tail_cut = min(np.percentile(g_pnl, 5), np.percentile(t_pnl, 5))
g_tail = g_pnl[g_pnl <= tail_cut]
t_tail = t_pnl[t_pnl <= tail_cut]
ax2.hist(g_tail, bins=120, density=True, alpha=0.55, color='steelblue',
         edgecolor='none', label='Gaussian MC')
ax2.hist(t_tail, bins=120, density=True, alpha=0.55, color='#d62728',
         edgecolor='none', label=f'Student-t MC (ν={df_t:.1f})')
ax2.axvline(-mc_results['es_99'], color='steelblue', lw=2, ls=':',
            label=f"Gauss 99% ES = {mc_results['es_99']:.4%}")
ax2.axvline(-t_results['es_99'], color='#d62728', lw=2, ls=':',
            label=f"t 99% ES = {t_results['es_99']:.4%}")
ax2.set_xlabel('Portfolio Return', fontsize=12)
ax2.set_ylabel('Density', fontsize=12)
ax2.set_title('Left-Tail Zoom — Fat-Tail Effect', fontsize=13, fontweight='bold')
ax2.legend(fontsize=9.5, loc='upper left')
ax2.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

fig.suptitle('Gaussian vs Student-t Monte Carlo — Impact of Heavy Tails',
             fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'figures' / 'gaussian_vs_student_t.png',
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

# Quantify the difference
print('\n99% Risk Metric Comparison: Gaussian vs Student-t')
print('=' * 55)
print(f"  {'Metric':<20} {'Gaussian':>12} {'Student-t':>12} {'Δ%':>10}")
print(f"  {'-'*20} {'-'*12} {'-'*12} {'-'*10}")
for label, gkey, tkey in [('VaR 99%', 'var_99', 'var_99'),
                           ('ES 99%', 'es_99', 'es_99')]:
    g_val = mc_results[gkey]
    t_val = t_results[tkey]
    pct = (t_val - g_val) / g_val * 100
    print(f"  {label:<20} {g_val:>11.4%} {t_val:>11.4%} {pct:>+9.1f}%")

### Observations on the Risk Model Comparison

**Key takeaways from the four-model comparison:**

1. **Parametric and Gaussian MC agree closely** — both assume Normal returns, so their VaR/ES estimates are similar. The small differences come from estimation noise vs analytical formula.

2. **Historical VaR is higher** — the empirical distribution captures fat tails and asymmetries that Gaussian models miss. This is expected: real market returns have excess kurtosis.

3. **Student-t MC produces the highest VaR and ES** — by explicitly modelling heavy tails via the fitted $\nu$ parameter, it assigns more probability to extreme events. The difference between Gaussian and Student-t ES is particularly notable, since ES is a tail-average measure.

4. **Basel III implication** — the regulatory shift from VaR to ES was motivated precisely by the fact that VaR ignores loss severity beyond the threshold. The Student-t ES provides a more conservative (and arguably more realistic) estimate of what happens in the worst scenarios.

---
## 5. Backtesting

### Rolling-Window Methodology

- **Window:** 250 trading days (~1 year)
- **Method:** Re-estimate μ and Σ at each step, compute 1-day 99% Parametric VaR
- **Validation:** Compare forecast to realized loss; record breaches

### Kupiec POF Test

$$LR = -2\ln\left[\frac{(1-p)^{T-x} \cdot p^x}{(1-\hat{p})^{T-x} \cdot \hat{p}^x}\right] \sim \chi^2(1)$$

$H_0$: The model is correctly specified (observed breach rate = expected breach rate)

In [ ]:
# Run full backtest
bt_results, breach_stats, kupiec = run_full_backtest(
    log_returns, weights, window=250, confidence_level=0.99
)

print('Breach Statistics')
print('=' * 45)
for k, v in breach_stats.items():
    if isinstance(v, float):
        print(f'  {k:.<35} {v:>10.4f}')
    else:
        print(f'  {k:.<35} {v:>10}')

print(f'\nKupiec POF Test')
print('=' * 45)
for k, v in kupiec.items():
    if isinstance(v, float):
        print(f'  {k:.<35} {v:>10.4f}')
    else:
        print(f'  {k:.<35} {str(v):>10}')

In [ ]:
# Rolling VaR vs Actual Losses
fig, ax = plt.subplots(figsize=(16, 7))

dates = bt_results['date']
var_line = bt_results['predicted_var']
losses = bt_results['actual_loss']
breaches = bt_results['breach']

ax.plot(dates, losses, color='steelblue', alpha=0.5, lw=0.8, label='Actual Daily Loss')
ax.plot(dates, var_line, color='#d62728', lw=1.5, label='99% VaR Forecast')

breach_dates = dates[breaches]
breach_losses = losses[breaches]
ax.scatter(breach_dates, breach_losses, color='#e74c3c', s=30, zorder=5,
           label=f'Breaches (n={breaches.sum()})')

ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)
ax.set_title('Rolling 99% VaR Backtest — Predicted vs Actual Losses',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
fig.autofmt_xdate()
plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'figures' / 'rolling_var_backtest.png',
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 6. Stress Testing

### Scenario 1: Volatility Shock
$$\Sigma_{\text{shock}} = 2\Sigma$$

Equivalent to doubling all asset volatilities — mimics a sudden market dislocation.

### Scenario 2: Correlation Stress
$$\rho_{ij} \rightarrow 0.9 \quad \forall \, i \neq j$$

Simulates complete diversification collapse under systemic stress — all assets move together.

In [ ]:
# Full stress analysis
stress_results = full_stress_analysis(
    mu, cov, weights, mc_results, NUM_SIMULATIONS, RANDOM_SEED
)

print('Volatility Shock Impact (2× Σ)')
print('=' * 55)
for k, v in stress_results['vol_shock']['impact'].items():
    if isinstance(v, float):
        print(f'  {k:.<40} {v:>12.4f}')

print(f'\nCorrelation Stress Impact (ρ = 0.9)')
print('=' * 55)
for k, v in stress_results['corr_stress']['impact'].items():
    if isinstance(v, float):
        print(f'  {k:.<40} {v:>12.4f}')

In [ ]:
# Stress comparison chart
fig, ax = plt.subplots(figsize=(12, 7))

metrics_keys = ['var_95', 'var_99', 'es_95', 'es_99']
labels = ['95% VaR', '99% VaR', '95% ES', '99% ES']
x = np.arange(len(metrics_keys))
width = 0.25

base_vals = [mc_metrics[f'mc_{m}'] for m in metrics_keys]
vol_vals = [stress_results['vol_shock']['results'][m] for m in metrics_keys]
corr_vals = [stress_results['corr_stress']['results'][m] for m in metrics_keys]

ax.bar(x - width, base_vals, width, label='Baseline', color='steelblue', alpha=0.8)
ax.bar(x, vol_vals, width, label='Vol Shock (2×)', color='#d62728', alpha=0.8)
ax.bar(x + width, corr_vals, width, label='Corr Stress (ρ=0.9)', color='#9467bd', alpha=0.8)

ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_xlabel('Risk Metric', fontsize=12)
ax.set_ylabel('Value', fontsize=12)
ax.set_title('Stress Test Comparison — Risk Metric Sensitivity',
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=11)
ax.legend(fontsize=11)
plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'figures' / 'stress_comparison.png',
            dpi=150, bbox_inches='tight', facecolor='white')
plt.show()

---
## 7. Model Limitations

| Limitation | Impact | Status |
|-----------|--------|--------|
| **Gaussian tails too thin** | Underestimates tail risk | ✅ **Addressed** — Student-t MC captures heavy tails |
| **Static correlation** | Misses regime-dependent correlation | ⬜ Future: DCC-GARCH or regime-switching |
| **No liquidity effects** | Overstates ability to exit positions | ⬜ Future: Liquidity-adjusted VaR |
| **No regime switching** | Misses volatility clustering | ⬜ Future: GARCH conditional volatility |
| **Single-period horizon** | √t scaling assumes i.i.d. | ⬜ Future: Multi-step MC simulation |
| **Linear portfolio** | Cannot handle options/derivatives | ⬜ Future: Delta-gamma or full revaluation |

The Student-t extension directly addresses the most impactful limitation (Gaussian underestimation of tail risk). The remaining items are standard trade-offs in first-generation desk-level risk models. Production systems at major institutions address these through more sophisticated statistical frameworks (DCC-GARCH, EVT, filtered historical simulation).

---
## 8. Methodology Summary

### Algorithm Pipeline

```
Historical Prices
    │
    ▼
Log Returns (r_t = ln(P_t / P_{t-1}))
    │
    ├──→ Mean Vector (μ)
    ├──→ Covariance Matrix (Σ)
    │
    ├──→ Historical VaR (empirical quantile)
    ├──→ Parametric VaR (z_α · σ_p - μ_p)
    ├──→ Gaussian Monte Carlo VaR
    │       ├── Cholesky: Σ = L L^T
    │       ├── Z ~ N(0, I)
    │       ├── R = μ + L Z
    │       └── VaR, ES from quantiles
    │
    └──→ Student-t Monte Carlo VaR
            ├── Fit ν via MLE
            ├── Z ~ t(ν), rescale: Z̃ = Z·√((ν-2)/ν)
            ├── R = μ + L Z̃
            └── VaR, ES from quantiles
                    │
                    ├──→ Backtesting (250-day rolling, Kupiec test)
                    └──→ Stress Testing (vol shock, corr stress)
```

### Assumptions

1. Log returns are stationary and ergodic
2. Historical covariance is a reasonable estimator of future risk
3. Portfolio is linear (no optionality)
4. Positions can be liquidated at market prices within 1 trading day
5. No transaction costs or market impact
6. Student-t degrees of freedom are stable over the sample period

---

*Report generated by Monte Carlo Market Risk Engine v2.0 — includes Student-t heavy-tailed extension*